<table width="100%">
    <tr>
      <td align="center" colspan=2><h1>DICOM Basics</h1></td>
    </tr>
</table>

Radiologists live and breathe DICOM, the standard file format for all diagnostic imaging studies.

DICOM files contain the actual pixel values for the image of course, but also a bunch of *metadata*, or data about data.

DICOM metadata encodes the context of the image - which patient it refers to, when and where it was obtained, how it relates to other images (i.e. as part of the same series or study), and some technical parameters that might be necessary to properly reconstruct the full image set.

```pydicom``` is a popular Python library for dealing with DICOM data. If we install it on our VM we can then import it and use its functions to manipulate DICOM.



In [ ]:
!pip3 install matplotlib
!pip3 install pydicom
!pip3 install scikit-image
!pip3 install pydicom

In [ ]:
%matplotlib inline

from pydicom import dcmread
from pydicom.multival import MultiValue
from pydicom.errors import InvalidDicomError
import matplotlib.pyplot as plt
import os
import platform
from skimage.transform import resize

In [ ]:
size = 256, 256
dicomdir = "./resources/dicoms"

In [ ]:
# from fastai2 medical imaging
def windowed(px, w, l):
    """Windows a pixel_array of Hounsfield units
    args:
      px = pixel array in Hounsfield units
      w = window width (HU range)
      l = window level (center point)
    returns:
      pixel_array convered to the given window/level
    """
    if type(w) == MultiValue:
      w = w[0]
    if type(l) == MultiValue:
      l = l[0]
    px_min = l - w//2
    px_max = l + w//2
    px[px<px_min] = px_min
    px[px>px_max] = px_max
    return (px-px_min) / (px_max-px_min) 

In [ ]:
# Read each of the files in dicomdir, print some metadata, and show the image
for root, dirs, files in os.walk(dicomdir):
  for f in sorted(files, reverse=True):
    try:
      ds = dcmread(os.path.join(dicomdir, f))
    except (PermissionError, InvalidDicomError, FileNotFoundError):
      continue
    
    im = ds.pixel_array
    im = im*ds.RescaleSlope + ds.RescaleIntercept
    im = windowed(im, ds.WindowWidth, ds.WindowCenter)
    if(ds.PhotometricInterpretation == 'MONOCHROME2'):
        cmap=plt.cm.bone
    elif(ds.PhotometricInterpretation == 'MONOCHROME1'):
        cmap=plt.cm.bone_r
    else:
        print("Unknown Photometric Interpretation")
        cmap=plt.cm.bone
        continue
    
    print()
    print("Filename..................:", f)
    print("Storage type..............:", ds.SOPClassUID)
    print()

    pat_name = ds.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient name..............:", display_name)
    print("Patient ID................:", ds.PatientID)
    print("Modality..................:", ds.Modality)
    print("Rescale Intercept.........:", ds.RescaleIntercept)
    print("Rescale Slope.............:", ds.RescaleSlope)
    
    if 'PixelData' in ds:
      rows = int(ds.Rows)
      cols = int(ds.Columns)
      print("Image size ...............: {rows:d} x {cols:d}, {size:d} bytes".format(
          rows=rows, cols=cols, size=len(ds.PixelData)))
      print("Photometric Interpretation:", ds.PhotometricInterpretation)
    
    plt.imshow(im, cmap="gray")  
    plt.show()

In [ ]:
# Look at the entire DICOM metadata header
ds

If you follow along in the code block above, you'll see that we read the pixel data from the DICOM file with 

```
im = ds.pixel_array
```

What IS a pixel array? And while we're at it, how is it that we can do math with images??

A pixel array is nothing more than numbers organized in rows and columns.

In fact, we can create an array, save it to a file, and open it directly in Excel! 

In [ ]:
import numpy as np
# Create a simple 5x5 array of 0s and 1s
ar = np.asarray(
    [
      [1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0],
      [0, 0, 1, 0, 0],
      [0, 1, 0, 1, 0],
      [1, 0, 0, 0, 1],
    ]
  ) 

np.savetxt('array.csv', ar, delimiter=",")

# See what happens when we resize it
size = (64, 64)
ar = resize(ar, size, mode='constant', cval=0, preserve_range=True)
np.savetxt('array_resized.csv', ar, delimiter=",")

In [ ]:
# We can do the same thing with the DICOM pixel array!
im = ds.pixel_array
#plt.hist(im.flatten(), bins=20)
#plt.show()
im = im*ds.RescaleSlope + ds.RescaleIntercept
im = resize(im, [128, 128], preserve_range=True)
#plt.hist(im.flatten(), bins=20)
#plt.show()
np.savetxt("dicom.csv", im, delimiter=",")

im = windowed(im, ds.WindowWidth, ds.WindowCenter)
#plt.hist(im.flatten(), bins=20)
#plt.show()
np.savetxt("dicom_windowed.csv", im, delimiter=",")

Thinking of an image as a spreadsheet containing organized rows and columns of numbers helps to de-mystify the idea of performing math operations on images - neural network operations on image data are no different than adding, averaging, or finding the maximum value in a range of Excel cells, and then creating new arrays from the results of those operations, peforming similar operations again, ad infinitum :)

You can actually implement a simple facial recognition algorithm (and any other type of neural network if you try hard enough) [completely in Excel](https://towardsdatascience.com/cutting-edge-face-recognition-is-complicated-these-spreadsheets-make-it-easier-e7864dbf0e1a)! 